# Creating the CSV File for NFT Ranks

---

This notebook generates a CSV file mapping NFTs to their ranks, which is later used in other notebooks such as [`calculations.ipynb`](./calculations.ipynb). While the `calculations.ipynb` notebook demonstrates the process using an existing CSV file, this notebook shows how to create that file from scratch.

Before running this notebook:
- Ensure you have the JSON files generated by [`download_nft_data.ipynb`](download_nft_data.ipynb) and [`count_trait_values.ipynb`](count_trait_values.ipynb).
- Run the cells in sequence to produce the final CSV file.


---
## 1.1 Open Statements

In [1]:
open System.Text.Json
open System.IO
open FSharp.Reflection

### 1.2 Fetching Data

In this step, we load the NFT data and trait value distributions from the JSON files generated in previous notebooks:
- `collection_1.json` and `collection_2.json` for NFT data.
- `collectiontraits_1.json` and `collectiontraits_2.json` for trait occurrences.

Ensure these files are present in the `../data/` directory before proceeding.

In [7]:
type Nft = {
    assetName: string
    encodedName: string
    Background: string
    Bear: string
    Clothes: string
    Face: string
    Handheld: string
    Head: string
    Skins: string
}

type NftTraits = {
    Background: Map<string, int>
    Bear: Map<string, int>
    Clothes: Map<string, int>
    Face: Map<string, int>
    Handheld: Map<string, int>
    Head: Map<string, int>
    Skins: Map<string, int>
}

In [8]:
let loadNfts<'T> (filePath: string) : 'T =
    let jsonContent = File.ReadAllText(filePath)
    JsonSerializer.Deserialize<'T>(jsonContent)

In [9]:
let nftRoundOneNfts = loadNfts<Nft list> "./data/tbc_roundone.json"
let nftRoundTwoNfts = loadNfts<Nft list> "./data/tbc_roundtwo.json"

In [10]:
let nftRoundOneTraits = loadNfts<NftTraits> "./data/tbc_traits_roundone.json"
let nftRoundTwoTraits = loadNfts<NftTraits> "./data/tbc_traits_roundtwo.json"

### 1.3 Calculating Trait Value Scores
Using the trait occurrence data, we calculate the rarity scores for each trait value. These scores are used to compute the overall rarity score for each NFT.

In [11]:
let rOneTotalNfts = 804.0
let rTwoTotalNfts = 2035.0

In [12]:
let processNftTraits (nftTraits: NftTraits) (totalNfts: float) =
    let recordType = typeof<NftTraits>
    FSharp.Reflection.FSharpType.GetRecordFields(recordType)
    |> Array.fold (fun acc field ->
        let fieldValue = field.GetValue(nftTraits) :?> Map<string, int>
        let traitScores = 
            fieldValue |> Map.map (fun key value ->
                Math.Round(100.0 - (float value / float totalNfts) * 100.0, 2)
            )
        Map.add field.Name traitScores acc
    ) Map.empty

In [13]:
let rOneScoredTraits = processNftTraits nftRoundOneTraits rOneTotalNfts
let rTwoScoredTraits = processNftTraits nftRoundTwoTraits rTwoTotalNfts

### 1.4 Compute NFT Ranks
This section assigns a rank to each NFT based on its rarity score. The ranks are calculated separately for Round 1 and Round 2 NFTs.

In [14]:
let computeNftRank (nftList: list<Nft>) nftTraitScores = 
    nftList
    |> List.map (fun nft ->
        let rarityScore =  
            [
                ("Background", nft.Background)
                ("Bear", nft.Bear)
                ("Clothes", nft.Clothes)
                ("Face", nft.Face)
                ("Handheld", nft.Handheld)
                ("Head", nft.Head)
                ("Skins", nft.Skins)
            ]
            |> List.fold (fun acc (attrName, attrValue) ->  
                match Map.tryFind attrName nftTraitScores with
                | Some traitMap ->  
                    acc + (Map.tryFind attrValue traitMap |> Option.defaultValue 0.0)
                | None -> acc 
            ) 0.0 
        (nft.assetName, nft.encodedName, rarityScore) 
    )
    |> List.sortBy (fun (_, _, rarityScore) -> -rarityScore)
    |> List.mapi (fun index (assetName, encodedName, rarityScore) -> 
    (index + 1, assetName, encodedName, rarityScore))

In [15]:
let rankedNftRoundOne = computeNftRank nftRoundOneNfts rOneScoredTraits
let rankedNftRoundTwo = computeNftRank nftRoundTwoNfts rTwoScoredTraits

#### CSV for Subject to Rank
This section creates the CSV file with columns `Name`, `Subject`, `Rank`, and `Score`.

In [16]:
let listToCsv (data: (int * string * string * float) list) (policyId: string) (filePath: string) =
    let isNewFile = not (File.Exists(filePath))
    
    let header = "Name,Subject,Rank,Score"
    let rows =
        data
        |> List.map (fun (rank, assetName, encodedName, score) ->  
            let subject = sprintf "%s%s" policyId encodedName  
            sprintf "%s,%s,%d,%f" assetName subject rank score
        )
    
    let existingRows = 
        if not isNewFile then
            File.ReadAllLines(filePath) |> Set.ofSeq
        else
            Set.empty

    let newRows = 
        rows
        |> List.filter (fun row -> not (existingRows.Contains row))
    
    if isNewFile then
        File.AppendAllLines(filePath, [header])

    if newRows.Length > 0 then
        File.AppendAllLines(filePath, newRows)

In [17]:
let policyIdRoundOne = "ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d87";
let policyIdRoundTwo = "da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c";

listToCsv rankedNftRoundOne  policyIdRoundOne "./data/tbc_ranked_by_rarity.csv"
listToCsv rankedNftRoundTwo policyIdRoundTwo "./data/tbc_ranked_by_rarity.csv"